$$
\begin{alignat*}{2}
\min \ & \sum_{i \in \mathcal{I}} f_i x_i + \sum_{s \in \mathcal{S}} p_s \left[ \sum_{i \in
\mathcal{I}} \sum_{j \in \mathcal{J}} c_{ij} y^s_{ij}  + \sum_{j \in \mathcal{J}} \lambda_j z^s_{j} \right] \\
\text{s.t.} \ &  \sum_{i \in \mathcal{I}} y^s_{ij} + z^s_{j} \geq d^s_{j} \quad && \forall j
\in \mathcal{J}, \  s \in \mathcal{S} \\
& \sum_{j \in \mathcal{J}} y^s_{ij} - u_i x_i \leq 0 \quad && \forall i \in
\mathcal{I}, \ s \in \mathcal{S} \\
& z^s_{j} \geq 0 && \forall j \in \mathcal{J}, \  s \in \mathcal{S} \\
& y^s_{ij} \geq 0 && \forall i \in \mathcal{I}, \  j \in \mathcal{J}, \  s \in \mathcal{S} \\
&x_{i} \in \{0,1\}  && \forall i \in \mathcal{I}
\end{alignat*}
$$

# Import Packages

In [ ]:
from gurobipy import *
import numpy as np

# Building the Data

In [ ]:
# Read the data from file
import FL_Data as Data

f = Data.f            # facility opening costs (j)
c = Data.c            # transportation costs (i,j)
Lambda = Data.Lambda  # penalty costs (j)
u = Data.u            # facility capacities (i)
d = Data.d            # demands (s,j)

In [ ]:
# Equally likely scenarios
nS = len(d)           # the number of scenarios
p = [1.0/nS] * nS     # scenario probabilities (assuming equally likely scenarios)

In [ ]:
# Build sets
I = range(len(c))
J = range(len(Lambda))
S = range(nS)

In [ ]:
# Build second-stage objective coefficients: Note that we scale them with scenario probabilities
Lambda_ForAllScen = [p[s] * Lambda[j] for j in J for s in S]
c_ForAllScen = [p[s] * c[i][j] for i in I for j in J for s in S]

# Start building the Model

In [ ]:
m = Model("2SP_ExtForm")

In [ ]:
# First-stage variables: facility openining decisions
x = m.addVars(I, vtype=GRB.BINARY, obj=f, name='x')

In [ ]:
# Second-stage vars: transportation and unmet demand decisions
y = m.addVars(I, J, S, obj=c_ForAllScen, name='y')
z = m.addVars(J, S, obj=Lambda_ForAllScen, name='z')

Note: Default variable bounds are LB = 0 and UB = infinity

In [ ]:
m.modelSense = GRB.MINIMIZE

In [ ]:
# Demand constraints
m.addConstrs(
  (y.sum('*',j,s) + z[j,s] == d[s][j] for j in J for s in S), name='Demand');

In [ ]:
# Production constraints
m.addConstrs(
  (y.sum(i,'*',s) <= u[i]*x[i] for i in I for s in S), name='Capacity');

# Solve the extensive form

In [ ]:
m.optimize()

In [ ]:
OptimalValue_2SP = m.objVal
print('\nEXPECTED COST : %g' % OptimalValue_2SP)

In [ ]:
print('SOLUTION:')
for i in I:
    if x[i].x > 0.99:
        print('Facility %s is open' % i)
        for j in J:
            if y[i,j,0].x > 0.00001:
                print('  Transport %g units to customer %s in scenario 0' % \
                      (y[i,j,0].x, j))

In [ ]:
print('AVERAGE UNMET DEMAND:')
for j in J:
    AvgUnmet = sum(z[j,s].x for s in S)/nS
    print('   Customer %s: %g' % (j, AvgUnmet))

# Build and solve the mean value problem

In [ ]:
# Compute average demands
d = np.array(d)
d_MV = np.sum(d,axis = 0)/nS

In [ ]:
m_MV = Model("MVmodel")

In [ ]:
# First-stage variables: facility opening decisions
x_MV = m_MV.addVars(I, vtype=GRB.BINARY, obj=f, name='x_MV')

In [ ]:
# Second-stage vars: transportation and unmet demand decisions
# Note that this is a single scenario problem, so we don't use the scenario index
y_MV = m_MV.addVars(I, J, obj=c, name='y_MV')
z_MV = m_MV.addVars(J, obj=Lambda, name='z_MV')

In [ ]:
m_MV.modelSense = GRB.MINIMIZE

In [ ]:
# Demand constraints
m_MV.addConstrs(
  (y_MV.sum('*',j) + z_MV[j] == d_MV[j] for j in J), 'Demand_MV');

In [ ]:
# Production constraints
m_MV.addConstrs(
  (y_MV.sum(i,'*') <= u[i]*x_MV[i] for i in I), 'Capacity_MV');

In [ ]:
## Solve ##
m_MV.optimize()

In [ ]:
print('\nMEAN VALUE FIRST-STAGE SOLUTION:')
for i in I:
    if x_MV[i].x > 0.99:
        print('Facility %s is open' % i)

In [ ]:
############# Fix this as first-stage solution in the stochastic model ##########
for i in I:
    if x_MV[i].x > 0.99:
        x[i].lb = 1.0
    else:
        x[i].ub = 0.0

In [ ]:
m.update()
m.optimize()

In [ ]:
print('\nEXPECTED COST of the MV SOLUTION: %g' % m.objVal)

In [ ]:
VSS = m.objVal - OptimalValue_2SP
print('\nVSS: %g' % VSS)